# Notebook de conception de Notebook

Ce Notebook .Net interactive a pour objectif de permettre la création assistée d'autres notebooks .Net interactive en confiant le soin à ChatGPT d'analyser et de proposer des modifications d'une version courante, et en prenant en charge la mise à jour et l'exécution des mises à jour en function calling Open AI grâce à l'API .Net interactive. 


### 1. Initialisation

On installe des packages pour la manipulation de notebook et pour l'orchestration de LLMs.

In [ ]:
// #r "nuget: Microsoft.DotNet.Interactive, *-*"
#r "nuget: Microsoft.DotNet.Interactive.CSharp, *-*"
#r "nuget: Microsoft.DotNet.Interactive.Documents, *-*"
#r "nuget: Microsoft.DotNet.Interactive.PackageManagement, *-*"


#r "nuget: Microsoft.Extensions.Logging"
#r "nuget: Microsoft.SemanticKernel, *-*"
#r "nuget: Microsoft.SemanticKernel.Planners.OpenAI, *-*"

- **Imports des espaces de noms**

On prend soin de distinguer le kernel d'exécution de notebook .Net interactive, et le kernel de semantic-kernel.

In [ ]:
  using Microsoft.DotNet.Interactive;
  using Microsoft.SemanticKernel;
  using Microsoft.SemanticKernel.Planning;
  using Microsoft.SemanticKernel.Connectors.OpenAI;
  
  using System;
  using System.IO;
  using System.Threading.Tasks;

  using SKernel = Microsoft.SemanticKernel.Kernel;
  using IKernel = Microsoft.DotNet.Interactive.Kernel;

- **Configurez l'authentification des services semantic-kernel**

Créer au besoin le fichier config/settings.json pour la config semantic-kernel

In [ ]:
// Load some helper functions, e.g. to load values from settings.json
#!import config/Settings.cs 

- **Création d'un logger dédié**

On crée un logger qui s'affichera en cellule de sortie

In [ ]:
using Microsoft.Extensions.Logging;
using System;

public class DisplayLogger : ILogger, ILoggerFactory
{
    private readonly string _categoryName;
    private readonly LogLevel _logLevel;

    public DisplayLogger(string categoryName, LogLevel logLevel)
    {
        _categoryName = categoryName;
        _logLevel = logLevel;
    }

    public IDisposable BeginScope<TState>(TState state) => this;

    public bool IsEnabled(LogLevel logLevel) => logLevel >= _logLevel;

    public void Log<TState>(LogLevel logLevel, EventId eventId, TState state, Exception exception, Func<TState, Exception, string> formatter)
    {
        if (!IsEnabled(logLevel))
        {
            return;
        }

        var logEntry = $"[{logLevel}] {_categoryName} - {formatter(state, exception)}";
        if (exception != null)
        {
            logEntry += Environment.NewLine + exception;
        }

        display(logEntry);
    }

        /// <inheritdoc/>
        public void Dispose()
        {
        // This class is marked as disposable to support the BeginScope method.
        // However, there is no need to dispose anything.
        }

    public ILogger CreateLogger(string categoryName) => this;

        public void AddProvider(ILoggerProvider provider) => throw new NotSupportedException();

}

public class DisplayLoggerProvider : ILoggerProvider
{
    private readonly LogLevel _logLevel;

    public DisplayLoggerProvider(LogLevel logLevel)
    {
        _logLevel = logLevel;
    }

    public ILogger CreateLogger(string categoryName)
    {
        return new DisplayLogger(categoryName, _logLevel);
    }

    public void Dispose() { }
}


- **Initialisation semantic-kernel:**

On utilise les informations de configuration de semantic-kernel pour créer un kernel. On le dotera par la suite de plugins pour l'utiliser dans un planner.

In [ ]:
using Microsoft.Extensions.Logging;
using Microsoft.Extensions.DependencyInjection;
using System;

// Configure AI service credentials used by the kernel
var (useAzureOpenAI, model, azureEndpoint, apiKey, orgId) = Settings.LoadFromFile();

// // Configure custom logger
// var loggerFactory = LoggerFactory.Create(builder =>
// {
//     builder.AddProvider(new DisplayLoggerProvider(LogLevel.Information));
// });

var builder = SKernel.CreateBuilder();

builder.Services.AddLogging(loggingBuilder =>
{
    loggingBuilder.AddProvider(new DisplayLoggerProvider(LogLevel.Information));
});

if (useAzureOpenAI)
    builder.AddAzureOpenAIChatCompletion(model, azureEndpoint, apiKey);
else
    builder.AddOpenAIChatCompletion(model, apiKey, orgId);

// builder.WithLoggerFactory(loggerFactory);

var semanticKernel = builder.Build();

display("Kernel and Semantic Kernel initialized.");


### 2. Mode de Fourniture des Informations

On permet à l'utilisateur de saisir les informations décrivant la tâche à accomplir dans le notebook de travail de plusieurs façons différentes.

In [ ]:
public enum InformationMode
{
    Variable,
    Prompt,
    File
}



### 3. Personnalisation du Notebook de Travail

On charge un notebook template contenant des parties de Markdown et de code à compléter, et on injecte la tâche dans la partie descriptive en entête du notebook .Net interactive.

In [ ]:
using System.IO;

public void SetStartingNotebook(string taskDescription, string notebookPath)
{
    var notebookTemplatePath = "./Workbook-Template.ipynb";
    // var notebookPath = @$"./Workbooks/Workbook-{DateTime.Now.Date.ToString("yyyy-MM-dd")}.ipynb";
    
    // var notebookOutputPath = @$"./Workbook-{DateTime.Now.Date.ToString("yyyy-MM-dd")}-output.ipynb";

    string notebookContent;
    if (!File.Exists(notebookPath))
    {
        notebookContent = File.ReadAllText(notebookTemplatePath);
    }
    else
    {
        notebookContent = File.ReadAllText(notebookPath);
    }


    display(@"Personnalisation du notebook {notebookPath} en cours...");

    notebookContent = notebookContent.Replace("{{TASK_DESCRIPTION}}", taskDescription);

    File.WriteAllText(notebookPath, notebookContent);
    display($"Notebook personnalisé prêt à l'exécution\n{notebookContent}");
}



### 4. Boucle Récurrente pour l'Exécution du Notebook

- **Classe en charge de charger, d'exécuter et renvoyer un le résultat d'un notebook**

In [ ]:
using Microsoft.DotNet.Interactive;
using Microsoft.DotNet.Interactive.Documents;
using KernelInfo = Microsoft.DotNet.Interactive.Documents.KernelInfo;
using Microsoft.DotNet.Interactive.Commands;
using Microsoft.DotNet.Interactive.Events;
using System.Reactive.Linq;
using System.Text;
using System.Threading;
using System.Collections.Generic;
using System.Linq;
using System.Threading.Tasks;

public class NotebookExecutor
{
    

    private readonly CompositeKernel _kernel;

    public NotebookExecutor(CompositeKernel kernel)
    {
        _kernel = kernel;
    }

    public async Task<InteractiveDocument> RunNotebookAsync(
        InteractiveDocument notebook,
        IDictionary<string, string>? parameters = null,
        CancellationToken cancellationToken = default)
    {
        var notebookExecutionDisplay = display("Exécution du notebook en cours...");
        notebookExecutionDisplay.Update("Début de l'exécution du notebook...");

        var resultDocument = new InteractiveDocument();

        if (parameters is not null)
        {
            parameters = new Dictionary<string, string>(parameters, StringComparer.InvariantCultureIgnoreCase);
        }

        var kernelInfoCollection = CreateKernelInfos(_kernel);
        var lookup = kernelInfoCollection.ToDictionary(k => k.Name, StringComparer.OrdinalIgnoreCase);

        foreach (var element in notebook.Elements)
        {
            if (lookup.TryGetValue(element.KernelName!, out var kernelInfo) &&
                StringComparer.OrdinalIgnoreCase.Equals(kernelInfo.LanguageName, "markdown"))
            {
                var formattedValue = new FormattedValue("text/markdown", element.Contents);
                var displayValue = new DisplayValue(formattedValue);
                display($"Affichage du markdown: \n{element.Contents}");
                await _kernel.SendAsync(displayValue);
                resultDocument.Add(element); 
            }
            else
            {
                try
                {
                    var submitCode = new SubmitCode(element.Contents, element.KernelName);
                    display($"Envoi du code au kernel {element.KernelName}:\n{element.Contents}");

                    KernelCommandResult codeResult = await _kernel.SendAsync(submitCode);

                    var outputs = new List<InteractiveDocumentOutputElement>();

                    foreach (var ev in codeResult.Events)
                    {
                        // display($"Événement reçu: {ev.GetType().Name}: {ev.ToString()}");
                        if (ev is DisplayEvent displayEvent)
                        {
                            outputs.Add(CreateDisplayOutputElement(displayEvent));
                        }
                        else if (ev is ErrorProduced errorProduced)
                        {
                            outputs.Add(CreateErrorOutputElement(errorProduced));
                        }
                        else if (ev is StandardOutputValueProduced stdOutput)
                        {
                            outputs.Add(new TextElement(stdOutput.Value.ToString(), "stdout"));
                        }
                        else if (ev is StandardErrorValueProduced stdError)
                        {
                            outputs.Add(new TextElement(stdError.Value.ToString(), "stderr"));
                        }
                        else  if (ev is CommandFailed commandFailed)
                        {
                            outputs.Add(CreateErrorOutputElement(commandFailed));
                        }
                        else if (ev is DisplayedValueProduced displayedValueProduced)
                        {
                             outputs.Add(CreateDisplayOutputElement(displayedValueProduced));
                        }
                        else if (ev is DisplayedValueUpdated displayedValueUpdated)
                        {
                             outputs.Add(CreateDisplayOutputElement(displayedValueUpdated));
                        }
                        else if (ev is ReturnValueProduced returnValueProduced)
                        {
                             outputs.Add(CreateDisplayOutputElement(returnValueProduced));
                        }
                        else if (ev is DiagnosticsProduced diagnosticsProduced)
                        {
                            diagnosticsProduced.Diagnostics
                                .Select(d => new ErrorElement(d.Severity.ToString(), d.Message))
                                .ToList()
                                .ForEach(e => outputs.Add(e));
                            //  outputs.Add(CreateDisplayOutputElement(returnValueProduced));
                        }
                        // else  if (ev is CommandSucceeded commandSucceeded)
                        // {
                        //     outputs.Add();
                        // }
                        // else  if (ev is CompleteCodeSubmissionReceived completeCodeSubmissionReceived)
                        // {
                        //     outputs.Add();
                        // }
                        else  
                        {
                            // display($"Événement non géré: {ev.GetType().Name}");
                        }
                        
                    }

                    var newElement = new InteractiveDocumentElement(
                        element.Contents,
                        element.KernelName,
                        outputs);

                    resultDocument.Add(newElement); // Ajout du résultat au document résultant
                }
                catch (Exception ex)
                {
                    display($"Crash du kernel {element.KernelName}");
                    var errorElement = new ErrorElement("Error", ex.Message);
                    var newElement = new InteractiveDocumentElement(                        
                        element.Contents, element.KernelName,
                        new List<InteractiveDocumentOutputElement> { errorElement });

                    resultDocument.Add(newElement); // Ajout du résultat au document résultant
                }
            }

        }

        var defaultKernelName = _kernel.DefaultKernelName;
        var defaultKernel = _kernel.ChildKernels.SingleOrDefault(k => k.Name == defaultKernelName);
        var languageName = defaultKernel?.KernelInfo.LanguageName ?? notebook.GetDefaultKernelName() ?? "C#";

        resultDocument.Metadata["kernelspec"] = new Dictionary<string, object>
        {
            { "name", defaultKernel?.Name ?? "csharp" },
            { "language", languageName }
        };

        notebookExecutionDisplay.Update("Exécution du notebook terminée.");

        return resultDocument;
    }

    private KernelInfoCollection CreateKernelInfos(CompositeKernel kernel)
    {
        KernelInfoCollection kernelInfos = new();

        foreach (var childKernel in kernel.ChildKernels)
        {
            kernelInfos.Add(new KernelInfo(childKernel.Name, languageName: childKernel.KernelInfo.LanguageName, aliases: childKernel.KernelInfo.Aliases));
        }

        if (!kernelInfos.Contains("markdown"))
        {
            kernelInfos = kernelInfos.Clone();
            kernelInfos.Add(new KernelInfo("markdown", languageName: "Markdown"));
        }

        return kernelInfos;
    }


    private DisplayElement CreateDisplayOutputElement(DisplayEvent displayEvent) =>
        new(displayEvent
            .FormattedValues
            .ToDictionary(
                v => v.MimeType,
                v => (object)v.Value));

    private ErrorElement CreateErrorOutputElement(ErrorProduced errorProduced) =>
        new(errorName: "Error",errorValue: errorProduced.Message);

    private ErrorElement CreateErrorOutputElement(CommandFailed failed) =>
    new(errorName: "Error",
        errorValue: failed.Message,
        stackTrace: failed.Exception switch
        {
            { } ex => SplitIntoLines(ex.StackTrace ?? ""),
            _ => Array.Empty<string>()
        });


    public static string PlainTextValue(DisplayEvent @event)
    {
        return @event.FormattedValues.FirstOrDefault()?.Value ?? string.Empty;
    }

     public static string[] SplitIntoLines(string s) =>
     s.Split(new[] { "\r\n", "\n" }, StringSplitOptions.None);
}


- **Classe du plugin semantic-kernel pour function calling**

Ce plugin permet à chat GPT de mettre à jour un notebook, et lui renvoie la nouvelle version rééxécutée.

In [ ]:
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Planning;
using Microsoft.DotNet.Interactive.Documents;
using Microsoft.DotNet.Interactive.Documents.Jupyter;
using System.Threading.Tasks;
using System.IO;
using System.ComponentModel;
using Microsoft.DotNet.Interactive.CSharp;
using System.Reflection;
using Microsoft.DotNet.Interactive.PackageManagement;

public class WorkbookInteraction
{
    private readonly string _notebookPath;
    private readonly NotebookExecutor _executor;
    private object _workbookDisplay;
    private MethodInfo _updateDisplayMethod = typeof(DisplayedValue).GetMethod("Update");
    private int _iterationCount = 0;

    private readonly ILogger _logger;

    public WorkbookInteraction(string notebookPath, ILogger logger)
    {
        _notebookPath = notebookPath;

        var cSharpKernel = new CSharpKernel()
            .UseKernelHelpers()
            .UseWho()
            .UseValueSharing();

        cSharpKernel.UseNugetDirective((k, resolvedPackageReference) =>
        {
            k.AddAssemblyReferences(resolvedPackageReference
                .SelectMany(r => r.AssemblyPaths));
            return Task.CompletedTask;
        }, false);

        var compositeKernel = new CompositeKernel
        {
            cSharpKernel
        };

        _executor = new NotebookExecutor(compositeKernel);
        _logger = logger;
    }

    private void DisplayWorkbook(string displayContent)
    {
        if (_workbookDisplay is null)
        {
            _workbookDisplay = display(displayContent);
        }

        _updateDisplayMethod.Invoke(_workbookDisplay, new object[] { displayContent });
    }

    [KernelFunction]
    [Description("Runs an updated version of the workbook and returns the notebook with output cells")]
    public async Task<string> UpdateEntireWorkbook(
        [Description("the new version of the workbook in ipynb json format, with multiple edited cells, and outputs optional")] string updatedWorkbook)
    {
        var updateDisplay = display($"Appel en function calling à UpdateWorkbook avec le notebook...\n{updatedWorkbook}");
        File.WriteAllText(_notebookPath, updatedWorkbook);

        try
        {
            var notebook = await InteractiveDocument.LoadAsync(new FileInfo(_notebookPath));
            var resultDocument = await _executor.RunNotebookAsync(notebook);
            var outputJson = resultDocument.ToJupyterJson();
            File.WriteAllText(_notebookPath, outputJson);
            display($"Appel à UpdateWorkbook terminé, renvoi du workbook après réexécution...\n{outputJson}");
            _iterationCount++;
            display($"WorkbookInteraction Itération {_iterationCount} terminée.");
            return outputJson;
        }
        catch (Exception ex)
        {
            var message = $"Erreur lors de l'exécution du notebook: {ex.Message}";
            display(message);
            _logger.LogError(ex, "Erreur lors de l'exécution du notebook");
            return message;
        }
    }

    [KernelFunction]
    [Description("Updates a specific cell in the workbook, and returns the notebook with output cells")]
    public async Task<string> UpdateWorkbookCell(
        [Description("A unique string that is part of the cell's content to update, to tell it apart from other cells")] string uniqueContent,
        [Description("The string to be replaced in the cell, or empty for appending the new content at a new line at the end of the cell")] string targetContent,
        [Description("The new string for replacement or addition")] string newContent)
    {
        var updateDisplay = display($"Appel en function calling à UpdateWorkbookCell avec le notebook...\nCell Identifier: {uniqueContent}\nTarget String: {targetContent}\nNew Content: {newContent}");

        try
        {
            var notebook = await InteractiveDocument.LoadAsync(new FileInfo(_notebookPath));
            var cell = notebook.Elements.FirstOrDefault(e => e.Contents.StartsWith(uniqueContent));
            if (cell == null)
            {
                throw new Exception($"Cell with identifier '{uniqueContent}' not found.");
            }

            if (string.IsNullOrEmpty(targetContent))
            {
                cell.Contents = $"{cell.Contents}\n{newContent}";
            }
            else
            {
                cell.Contents = cell.Contents.Replace(targetContent, newContent);
            }

            
            var resultDocument = await _executor.RunNotebookAsync(notebook);
            var outputJson = resultDocument.ToJupyterJson();
            File.WriteAllText(_notebookPath, outputJson);
            display($"Appel à UpdateWorkbookCell terminé, renvoi du workbook après réexécution...\n{outputJson}");
            _iterationCount++;
            display($"WorkbookInteraction Itération {_iterationCount} terminée.");
            return outputJson;
        }
        catch (Exception ex)
        {
            var message = $"Erreur lors de l'exécution du notebook: {ex.Message}";
            display(message);
            _logger.LogError(ex, "Erreur lors de l'exécution du notebook");
            return message;
        }
    }
}


- **Classe en charge de l'exécution du planner:**

In [ ]:
#pragma warning disable SKEXP0060

public class NotebookUpdater
{
    private readonly FunctionCallingStepwisePlanner _planner;
    private readonly SKernel _semanticKernel;
    private readonly string _notebookPath;
    private readonly ILogger _logger;
    

    public NotebookUpdater(SKernel semanticKernel, string notebookPath, int maxIerations = 5, ILogger logger = null)
    {
        _semanticKernel = semanticKernel;
        _notebookPath = notebookPath;
        var options = new FunctionCallingStepwisePlannerOptions
        {
            MaxTokens = 20000,
            MaxTokensRatio = 0.2,
            MaxIterations = maxIerations,
            ExecutionSettings = new OpenAIPromptExecutionSettings { 
                ToolCallBehavior = ToolCallBehavior.AutoInvokeKernelFunctions
                 }
        };
        _planner = new FunctionCallingStepwisePlanner(options);
        _logger = logger;
        
        var workbookInteraction = new WorkbookInteraction(notebookPath, _logger);
        _semanticKernel.ImportPluginFromObject(workbookInteraction);
    }

    public async Task<string> UpdateNotebook()
    {
        display("Lecture du contenu du notebook...");
        var notebookJson = File.ReadAllText(_notebookPath);

        var updateDisplay = display("Appel de ChatGPT avec le workbook initialisé...");
        
        var plannerPrompt = $"Analyse le notebook suivant qui contient la description de son objectif, utilise le function calling avec la méthode UpdateWorkbook pour éditer et réexécuter le notebook jusqu'à ce qu'il donne satisfaction et renvoie la réponse finale. N'hésite pas à éditer plusieurs cellules de front mais priorise les erreurs qui apparaissent dans les cellules de sorties pour garder le notebook fonctionnel. Fais particulièrement à ne pas halluciner des packages Nuget non existants.\n\n{notebookJson}";
        display($"Envoi du prompt au planner...\n{plannerPrompt}");
        

        var result = await _planner.ExecuteAsync(_semanticKernel, plannerPrompt);
        
        
        
        updateDisplay.Update("Notebook mis à jour avec succès.");

        return result.FinalAnswer;
    }
}
